In [ ]:
# ---- Install dependencies if needed ----
!pip install flask scikit-learn pandas --quiet

# ---- Imports ----
from flask import Flask, request, jsonify, render_template_string
import pickle, re, numpy as np, pandas as pd
from collections import deque
from datetime import datetime
import threading
import webbrowser

# ---- Load Model + Vectorizer ----
with open("model.pkl","rb") as f:
    model = pickle.load(f)
with open("vectorizer.pkl","rb") as f:
    vectorizer = pickle.load(f)

# ---- Flask App ----
app = Flask(__name__)
history = deque(maxlen=50)

# ---- HTML + CSS + JS ----
HTML = r"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Cyber Abuse Detector</title>
<link href="https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&display=swap" rel="stylesheet">
<style>
body {
    margin:0; padding:0;
    min-height:100vh; display:flex; justify-content:center; align-items:center;
    background: linear-gradient(to bottom, #0b0033, #000010);
    font-family:'Orbitron',sans-serif; color:#fce4ec; overflow:hidden;
}
/* Magical Falling Stars */
.star {
    position:absolute; border-radius:50%; background:white; opacity:0.8;
    animation-name: starFall; animation-timing-function: linear; animation-iteration-count: infinite;
}
@keyframes starFall {
    0% {transform: translateY(0px) translateX(0); opacity:0.8;}
    100% {transform: translateY(110vh) translateX(50px); opacity:0;}
}
@keyframes twinkle {
    0%,100% {opacity:0.3;}
    50% {opacity:1;}
}
.card {
    background: rgba(30,10,40,0.85);
    padding:40px; border-radius:25px;
    width:90%; max-width:600px; box-shadow:0 0 50px #f48fb1;
    text-align:center; position:relative; z-index:1;
}
h2 {
    font-size:50px; font-weight:900; margin-bottom:25px;
    color:#f06292; text-shadow:0 0 25px #f48fb1,0 0 50px #f48fb1,0 0 75px #f48fb1;
    text-align:center;
}
textarea {
    width:100%; min-height:140px; border-radius:12px; padding:14px; font-size:16px;
    border:1px solid #f48fb1; background:#2a1a2f; color:#fce4ec; resize:none;
}
textarea:focus {border-color:#f06292; outline:none; background:#331f3f;}
.buttons {display:flex; flex-direction:column; gap:12px; margin-top:18px;}
button {
    width:100%; padding:14px; border:none; border-radius:12px; font-weight:700; cursor:pointer;
    font-size:16px; position:relative; overflow:hidden; color:#000; background:#f48fb1;
    box-shadow:0 0 10px #f48fb1, 0 0 20px #f48fb1;
    animation: neonPulse 2s infinite alternate;
}
#resetBtn {background:#f06292;}
#exportBtn {background:#ec407a;}
@keyframes neonPulse {
    0% {box-shadow:0 0 5px #f48fb1,0 0 15px #f48fb1;}
    50% {box-shadow:0 0 20px #f48fb1,0 0 40px #f48fb1;}
    100% {box-shadow:0 0 10px #f48fb1,0 0 25px #f48fb1;}
}
.result {margin-top:20px; padding:14px; border-radius:12px; font-weight:700; text-align:center; display:none; opacity:0; transition:opacity 0.5s; font-size:16px;}
.result.show {opacity:1;}
.abuse {background:#f06292; color:#000;}
.safe {background:#f8bbd0; color:#000;}
.confidence-bar {height:12px; border-radius:9px; margin-top:14px; overflow:hidden; background:#222;}
.confidence-fill {height:100%; width:0%; border-radius:9px; transition:width 0.8s ease;}
.history {margin-top:20px; max-height:220px; overflow-y:auto; border:1px solid #f48fb1; border-radius:12px; padding:10px; opacity:0; transition:opacity 0.5s; font-size:14px;}
.history.show {opacity:1;}
.history-item {margin-bottom:8px; padding:8px; border-radius:10px; background:#2a1a2f;}
/* Mobile responsiveness */
@media (max-width: 600px){
    h2 {font-size:36px;}
    textarea {min-height:120px;}
    .card {padding:20px;}
    button {font-size:14px; padding:12px;}
}
</style>
</head>
<body>
<div id="starsContainer"></div>
<div class="card">
<h2>Cyber Abuse Detector</h2>
<textarea id="text" placeholder="Type Roman Urdu comment..."></textarea>
<div class="buttons">
<button id="analyzeBtn">Analyze</button>
<button id="resetBtn">Reset</button>
<button id="exportBtn">Export CSV</button>
</div>
<div id="result" class="result"></div>
<div class="confidence-bar"><div id="confidenceFill" class="confidence-fill"></div></div>
<div id="riskyWords" class="mt-2"></div>
<div id="history" class="history"></div>
</div>
<script>
// Stars animation
const starsContainer=document.getElementById("starsContainer");
for(let i=0;i<100;i++){
    let star=document.createElement("div"); star.className="star";
    let size=Math.random()*3+1; star.style.width=size+"px"; star.style.height=size+"px";
    star.style.left=Math.random()*100+"%"; star.style.top=Math.random()*-20+"px";
    star.style.animationDuration=(Math.random()*7+3)+"s"; star.style.animationDelay=(Math.random()*5)+"s";
    star.style.boxShadow="0 0 "+(size*4)+"px #fff";
    star.style.animationName="starFall, twinkle";
    star.style.animationDuration=(Math.random()*7+5)+"s, "+(Math.random()*2+1)+"s";
    starsContainer.appendChild(star);
}
// Buttons
const textArea=document.getElementById("text");
const analyzeBtn=document.getElementById("analyzeBtn");
const resetBtn=document.getElementById("resetBtn");
const exportBtn=document.getElementById("exportBtn");
analyzeBtn.addEventListener("click", async ()=>{
    let text=textArea.value.trim(); if(!text) return;
    let res=await fetch("/",{method:"POST",headers:{"Content-Type":"application/json"},body:JSON.stringify({text:text})});
    let data=await res.json();
    let box=document.getElementById("result"); 
    let risky=document.getElementById("riskyWords");
    let historyDiv=document.getElementById("history"); 
    let conf=document.getElementById("confidenceFill");
    box.style.display="block"; box.classList.add("show"); risky.innerHTML=""; 
    historyDiv.innerHTML=""; historyDiv.classList.add("show");
    if(data.prediction==="CA"){box.className="result abuse show"; box.innerHTML="⚠ Cyber Abuse Detected";}
    else{box.className="result safe show"; box.innerHTML="✔ Safe Comment";}
    conf.style.width=data.probability+"%";
    conf.style.background=data.prediction==="CA"?"#f06292":"#f48fb1";
    if(data.risky_words.length>0) risky.innerHTML="<b>Top Risky Words:</b> "+data.risky_words.join(", ");
    data.history.forEach(item=>{
        let div=document.createElement("div"); div.className="history-item";
        div.innerHTML=`<b>[${item.timestamp}]</b> ${item.text}<br><b>Prediction:</b> ${item.prediction} (${item.probability}%)`;
        historyDiv.appendChild(div);
    });
});
resetBtn.addEventListener("click",()=>{
    textArea.value=""; 
    let box=document.getElementById("result"); box.style.display="none"; box.classList.remove("show");
    document.getElementById("riskyWords").innerHTML=""; 
    let hist=document.getElementById("history"); hist.innerHTML=""; hist.classList.remove("show");
    document.getElementById("confidenceFill").style.width="0%";
});
exportBtn.addEventListener("click",()=>{
    fetch("/export_csv").then(r=>r.blob()).then(blob=>{
        const url=window.URL.createObjectURL(blob); 
        const a=document.createElement("a"); 
        a.href=url; a.download="cyber_abuse_history.csv"; document.body.appendChild(a); a.click(); a.remove();
    });
});
</script>
</body>
</html>
"""

# ---- Flask Routes ----
@app.route("/", methods=["GET","POST"])
def home():
    global history
    if request.method=="GET":
        return render_template_string(HTML)
    data=request.get_json()
    text=data.get("text","").strip()
    vec=vectorizer.transform([text])
    pred=model.predict(vec)[0]
    prob=0
    if hasattr(model,"predict_proba"):
        try:
            prob_arr=model.predict_proba(vec)[0]
            prob=round(float(np.max(prob_arr))*100,2)
        except:
            prob=0
    risky_words=[]
    if pred=="CA":
        words=re.findall(r'\w+',text.lower())
        for w,score in zip(vectorizer.get_feature_names_out(), vec.toarray()[0]):
            if w in words and score>0: risky_words.append(w)
        risky_words=sorted(risky_words,key=lambda x: vec.toarray()[0][vectorizer.vocabulary_.get(x,0)], reverse=True)[:3]
    timestamp=datetime.now().strftime("%H:%M:%S")
    history.appendleft({"text":text,"prediction":pred,"probability":prob,"timestamp":timestamp,"risky_words":risky_words})
    return jsonify({"prediction":pred,"probability":prob,"risky_words":risky_words,"history":list(history)})

@app.route("/export_csv")
def export_csv():
    df=pd.DataFrame(list(history))
    return df.to_csv(index=False),200,{"Content-Disposition":"attachment; filename=cyber_abuse_history.csv","Content-Type":"text/csv"}

# ---- Run Flask locally (works in browser) ----
if __name__=="__main__":
    webbrowser.open("http://127.0.0.1:5000")
    app.run(host="0.0.0.0", port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.43:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2025 01:32:28] "GET / HTTP/1.1" 200 -
